# weibo 크롤링

## 크롤링 조건 설정

In [12]:
# 페이지 스크롤 다운 횟수 설정(숫자가 클수록 많은 페이지 크롤링)
num_scrolldown = 2
# 날짜 설정 : "설정 날짜" 포함, 이후 날짜 데이터 출력됨.
start_date = '2022-01-01'

In [9]:
from bs4 import BeautifulSoup as bs
import time
from selenium import webdriver
import pandas as pd
from datetime import datetime

In [10]:
# 사이트 로그인
driver = webdriver.Chrome('chromedriver99')
driver.get("https://weibo.com/login.php")
time.sleep(5)
driver.implicitly_wait(15)

# 창 최대화
driver.maximize_window()
time.sleep(1)

username = 'globalgg1@gg.go.kr'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
element_id.send_keys(username)
time.sleep(1)
element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
element_password.send_keys(password)

time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()
time.sleep(5)

<ipython-input-10-fd217827f503>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('chromedriver99')
<ipython-input-10-fd217827f503>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginname"]')
<ipython-input-10-fd217827f503>:17: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[2]/div/input')
<ipython-input-10-fd217827f503>:23: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  driver.find_element_by_xpath('//*[@id="pl_login_form"]/div/div[3]/div[6]/a').click()


In [13]:
url = 'https://weibo.com/p/1001066011831795/home?from=page_100106&mod=TAB#place'
# webdriver 로드
driver.get(url)    
time.sleep(5)
driver.implicitly_wait(5)

# 페이지 스크롤 다운
for i in range(num_scrolldown):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(1)
    driver.implicitly_wait(5)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
print("타이틀 : ", soup.title.text)

# 타이틀 수집
raw_contents = soup.find_all('div', class_='WB_text W_f14')
content_list = []
num = len(raw_contents)
for i in range(num):
    content = raw_contents[i].text.strip()
    content_list.append(content)
print('원본 게시글 수 : ', len(raw_contents))
print('게시글 수 : ', len(content_list))

# 조회수 수집
raw_number_view = soup.find_all('i', class_='S_txt2')
number_view_list = []
for i in range(len(raw_number_view)):
    temp = raw_number_view[i].attrs['title']
    number_view = int(temp[9:-1])
    number_view_list.append(number_view)
print('조회수 리스트 원소수 : ', len(number_view_list))

# 정보 수집
raw_info = soup.find_all('span', 'line S_line1')
# 좋아요 수 수집
number_like_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 1
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    number_like = int(temp)
    number_like_list.append(number_like)
print('좋아요 리스트 원소수 : ', len(number_like_list))

# 공유 수 수집
number_share_list = []
num_range = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range):
    num = ( i + 1 ) * 4 - 3
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '转发' in temp:
        number_share_list.append(0)
    else:
        number_like = int(temp)
        number_share_list.append(number_like)
print('공유수 리스트 원소수 : ', len(number_share_list))

# 댓글수 수집
number_comment_list = []
num_range_like = (len(raw_info) - 3) // 4 + 1
raw_info = soup.find_all('span', 'line S_line1')
for i in range(num_range_like):
    num = ( i + 1 ) * 4 - 2
    temp = raw_info[num]
    temp = temp.find_all('em')[-1].text
    if '评论' in temp:
        number_comment_list.append(0)
    else:
        number_like = int(temp)
        number_comment_list.append(number_like)
print('댓글수 리스트 원소수 : ', len(number_comment_list))

# 게시글 날짜 수집
date_list = []
raw_date = soup.find_all('div', 'WB_from S_txt2')
for i in range(len(raw_date)):
    temp = raw_date[i].find('a', 'S_txt2')
    date = temp.attrs['title'][:10]
    date_list.append(date)
print('날짜 리스트 원소수 : ', len(date_list))

# 팔로워 수 수집
number_follower_list = []
number_of_follower = soup.find_all('strong', 'W_f16')
num = int(number_of_follower[1].text)
number_follower_list.append(num)
for i in range(len(date_list)-1):
    number_follower_list.append(0)
print('팔로워 수 리스트 원소수 : ', len(number_follower_list))

# 링크 수집
link_list = []
raw_links = soup.find_all('div', 'WB_from S_txt2')
for link in raw_links:
    temp = 'https://weibo.com' + link.find('a' ,'S_txt2').attrs['href']
    link_list.append(temp)
print("링크 리스트 원소수 : ", len(link_list))

타이틀 :  京畿道官博的微博_微博
원본 게시글 수 :  45
게시글 수 :  45
조회수 리스트 원소수 :  45
좋아요 리스트 원소수 :  45
공유수 리스트 원소수 :  45
댓글수 리스트 원소수 :  45
날짜 리스트 원소수 :  45
팔로워 수 리스트 원소수 :  45


In [30]:
# 데이터 프레임 생성
df = {'날짜':date_list,
    '제목':content_list,
    '조회수':number_view_list,
    '좋아요_수':number_like_list,
    '공유수':number_share_list,
    '댓글수':number_comment_list,
    '팔로워_수':number_follower_list,
    '링크':link_list}
data = pd.DataFrame(df)
data

날짜                                                 제목    조회수  \
0   2022-03-12  华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿。...   3457   
1   2022-03-10  走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花...   3058   
2   2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...   3912   
3   2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...   5076   
4   2022-03-02  京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...   8664   
5   2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...   7826   
6   2022-02-24  历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...  14879   
7   2022-02-22  世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...   9743   
8   2022-02-18  京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...  11812   
9   2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  15428   
10  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  15156   
11  2022-02-08  介绍一下京畿道安山市家族旅行路线。大韩民国安山市作为外国人居住众多的城市，有着以首都圈内最好...  16327   
12  2022-02-06  保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市。从很早的先...  17567   
13  2022-02-04  这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市。作为随处都可以感受到文化艺术的幸福...  16989   
14  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  17253   
15  2022-01-27  介绍一下京畿道观光名地，幸州山城与历史公园。在大韩民国京畿道有一些历史悠久的山城。其中，介绍...  20534   
16  2022-01-26  拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’。通过史无前例的灯光...  18378   
17  2022-01-24  主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园。位于京畿道高阳市的，现代汽...  19130   
18  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19122   
19  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  17771   
20  2022-01-15  MZ世代的元宇宙元宇宙图书馆，京畿道城南教育图书馆在京畿道有了元宇宙元宇宙图书馆。💚💚(大声...  16614   
21  2022-01-13  京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...  15698   
22  2022-01-07  🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...  15943   
23  2022-01-06  美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...  15209   
24  2022-01-05  京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...  13710   
25  2022-01-01               🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年。💕 ​​​​  11772   
26  2021-12-31  已经是12月31日了，2021年的最后一天😌 是时候在美丽的日落中回顾这一年并清除您的思绪，...  11367   
27  2021-12-27  可以化解冬天寒冷的温泉！ ♨明年冬天来韩国，一定要去京畿道利川逛逛。一起来探访韩国第一家德式...  11824   
28  2021-12-24  ❄️满满的冬季浪漫~圣诞节快到了！ 🎅💝🙋请投票选出您想去的京畿道圣诞冬季景点吧！🎄利川Si...  12162   
29  2021-12-22  💕京畿道的听力障碍人数与一般游客相同，也可以，享受高满意度的旅游服务啦！京畿道政府，正在努力...  10078   
30  2021-12-20  💏京畿道抱川的香草岛乐园，正在进行“光之童话节”✨📷度过一个温暖的冬天，在闪烁的星光下与心爱...   8679   
31  2021-12-17  旧加平站重生！ 🤗🎵名为#音乐站1939#的音乐综合体相见旧加平站，照片焕然一新❣#旧加平站...  21927   
32  2021-12-15  冬天不容错过的刺激活动，雪橇😆投票选出你想去的京畿道雪橇场！❄️高阳ONEMOUNT雪地公园...  19223   
33  2021-12-13  ♨️饱腹暖胃解寒去京畿道传统市场吧😋😋✔ 水原池洞市场 - 拥有100多年历史的水原代表性传...  30249   
34  2021-12-10  被称为韩国#部队火锅#的故乡京畿道议政府！ 在聚集了许多部队火锅店的#议政府部队火锅街#上，...  20268   
35  2021-12-08  水原超火爆高档烤排骨餐厅——佳浦亭 🥩水原的排骨出名，那是由来已久的事了。虽然说法纷纭，据说...  21952   
36  2021-12-06  我在哪里可以认识可爱的朋友？▶ 京畿道宠物收养中心🐶🐱提供收养和教育弃犬的场所提高对被遗弃动...   7847   
37  2021-12-03  京畿道为市民提供方便、准确的公共文件访问途径。我们正在致力于“京畿道公共语言替代语言”！ 😊...   7138   
38  2021-12-01  相信热爱看韩剧的大家都对韩国烤肉一点也不陌生，那大家来韩国旅行时又是否会去吃韩国的烤牛肉呢？...  19566   
39  2021-11-30  👭与心爱的人一起赏秋景京畿道，约会的好地方！ 🏻下次去韩国旅行想参观什么？请投票给京畿道山城...  20187   
40  2021-11-26  2021京畿世界陶瓷双年展，正在京畿道进行中位于京畿道光州的京畿陶瓷博物馆也在举办特别展览。...  18960   
41  2021-11-24  🍁 赏红叶🖼 一个可以享受文化生活的地方👉 京畿道龙仁的,湖岩美术馆 👈📍 京畿道龙仁市天仁...  16513   
42  2021-11-22  🌾 京畿道推荐旅游目的地，您可以充分享受秋天的蒲苇🌾明年秋天来韩国，就来京畿道赏蒲苇风光吧！...  16833   
43  2021-11-19  ☁️秋天，凉爽的天气里散步的好季节，任何人都可以轻松步行在京畿道的街道！ 🚶‍🚶✌️我们一起...  18501   
44  2021-11-17  为了改善并促长国家的生育率，京畿道内所有生育家庭将获得50万韩元的支援以缓解生育家庭的经济负...  16513   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
0       2    0    0  531039  https://weibo.com/6011831795/LjnLVmwNf?from=pa...  
1       5    0    2       0  https://weibo.com/6011831795/Lj5cR3FIu?from=pa...  
2       2    0    0       0  https://weibo.com/6011831795/LiNnAlUAv?from=pa...  
3       2    1    0       0  https://weibo.com/6011831795/LibISiazA?from=pa...  
4       1    0    0       0  https://weibo.com/6011831795/LhSpqqt1c?from=pa...  
5       1    0    0       0  

In [27]:
# 데이터 셋 날짜로 슬라이싱
slicing_data = data[data['날짜'] >= start_date]
slicing_data

날짜                                                 제목    조회수  \
0   2022-03-12  华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿。...   3457   
1   2022-03-10  走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花...   3058   
2   2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...   3912   
3   2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...   5076   
4   2022-03-02  京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...   8664   
5   2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...   7826   
6   2022-02-24  历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...  14879   
7   2022-02-22  世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...   9743   
8   2022-02-18  京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...  11812   
9   2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  15428   
10  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  15156   
11  2022-02-08  介绍一下京畿道安山市家族旅行路线。大韩民国安山市作为外国人居住众多的城市，有着以首都圈内最好...  16327   
12  2022-02-06  保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市。从很早的先...  17567   
13  2022-02-04  这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市。作为随处都可以感受到文化艺术的幸福...  16989   
14  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  17253   
15  2022-01-27  介绍一下京畿道观光名地，幸州山城与历史公园。在大韩民国京畿道有一些历史悠久的山城。其中，介绍...  20534   
16  2022-01-26  拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’。通过史无前例的灯光...  18378   
17  2022-01-24  主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园。位于京畿道高阳市的，现代汽...  19130   
18  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19122   
19  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  17771   
20  2022-01-15  MZ世代的元宇宙元宇宙图书馆，京畿道城南教育图书馆在京畿道有了元宇宙元宇宙图书馆。💚💚(大声...  16614   
21  2022-01-13  京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...  15698   
22  2022-01-07  🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...  15943   
23  2022-01-06  美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...  15209   
24  2022-01-05  京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...  13710   
25  2022-01-01               🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年。💕 ​​​​  11772   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
0       2    0    0  531039  https://weibo.com/6011831795/LjnLVmwNf?from=pa...  
1       5    0    2       0  https://weibo.com/6011831795/Lj5cR3FIu?from=pa...  
2       2    0    0       0  https://weibo.com/6011831795/LiNnAlUAv?from=pa...  
3       2    1    0       0  https://weibo.com/6011831795/LibISiazA?from=pa...  
4       1    0    0       0  https://weibo.com/6011831795/LhSpqqt1c?from=pa...  
5       1    0    0       0  https://weibo.com/6011831795/LhzKTe0Ht?from=pa...  
6       3    0    0       0  https://weibo.com/6011831795/LgXkk2CxN?from=pa...  
7       2    0    0       0  https://weibo.com/6011831795/LgF5WsoN4?from=pa...  
8       2    0    1       0  https://weibo.com/6011831795/Lg2AoaSKq?from=pa...  
9       2    1    0       0  https://weibo.com/6011831795/LfuYSiaEG?from=pa...  
10      4    0    0       0  https://weibo.com/6011831795/LeNtbwHoH?from=pa...  
11      3    0    1       0  https://weibo.com/6011831795/LeurruFMg?from=pa...  
12      4    0    0       0  https://weibo.com/6011831795/Le88pcEuW?from=pa...  
13      3    0    0       0  https://weibo.com/6011831795/LdVR00Ijh?from=pa...  
14      5    0    1       0  https://weibo.com/6011831795/Ldnc606Rh?from=pa...  
15      1    0    1       0  https://weibo.com/6011831795/LcBRrAZvy?from=pa...  
16      3    1    1       0  https://weibo.com/6011831795/Lcyygmjkp?from=pa...  
17      2    0    0       0  https://weibo.com/6011831795/LcdTRzXmm?from=pa...  
18      3    1    0       0  https://weibo.com/6011831795/LbCqQmNzF?from=pa...  
19      5    1    1       0  https://weibo.com/6011831795/Lbck79XJz?from=pa...  
20      3    0    0       0  https://weibo.com/6011831795/LaMlxesju?from=pa...  
21      5    1    1       0  https://weibo.com/6011831795/LaAyPh9ZV?from=pa...  
22      4    0    1       0  https://weibo.com/6011831795/L9FbKuzBl?from=pa...  
23      5    0 

In [28]:
# 인덱스 내림차순 정렬
final_data = slicing_data.sort_index(ascending=False)
final_data

날짜                                                 제목    조회수  \
25  2022-01-01               🎊 新年快乐! 🎊祝你2022年与京畿道一起度过幸福的一年。💕 ​​​​  11772   
24  2022-01-05  京畿道支援的异色创业趋势！充分利用共享厨房的 “G-创食学院”的运营共享厨房是指多个营业者共...  13710   
23  2022-01-06  美味的冬季京畿道传统市场一到冬天就会想起鲫鱼饼、烤红薯、鱼糕、甜饼等很多零食吧？请来访问具有...  15209   
22  2022-01-07  🍅京畿道光州市代表农作物番茄的重新诞生！酸酸甜甜的番茄辣椒酱同大韩民国代表酱料辣椒酱一样受到...  15943   
21  2022-01-13  京畿道为外国人居民定居支援的事业募集📣京畿道，支援外国人居民与高丽人同胞支援事业🔥在大韩民国...  15698   
20  2022-01-15  MZ世代的元宇宙元宇宙图书馆，京畿道城南教育图书馆在京畿道有了元宇宙元宇宙图书馆。💚💚(大声...  16614   
19  2022-01-17  京畿道平泽市，2021平泽世界文化周成功落下帷幕🎉连接世界的文化散策城市 🏃在京畿道平泽市举...  17771   
18  2022-01-20  🎹 风乐响起来!! 🎺🎺在京畿道的博物馆都去过哪里？@@(学习历史最好的地方👍👍)👉京畿道博...  19122   
17  2022-01-24  主题公园，都去过哪里呢介绍一下位于京畿道的国内最大的汽车主题公园。位于京畿道高阳市的，现代汽...  19130   
16  2022-01-26  拥有很多世界文化遗产的美丽的水原华城2021年在华城举办了‘媒体艺术秀’。通过史无前例的灯光...  18378   
15  2022-01-27  介绍一下京畿道观光名地，幸州山城与历史公园。在大韩民国京畿道有一些历史悠久的山城。其中，介绍...  20534   
14  2022-02-01  虎年大吉大吉大利新年蒙福#京畿道##朝鲜半岛##京畿道旅游##虎年大吉##大吉大利##新年蒙...  17253   
13  2022-02-04  这里是被得天独厚的自然环境包围的美丽的城市，京畿道果川市。作为随处都可以感受到文化艺术的幸福...  16989   
12  2022-02-06  保存着历史的自豪的城市富川，大韩民国历史旅行路线京畿道富川市是具有独特个性的城市。从很早的先...  17567   
11  2022-02-08  介绍一下京畿道安山市家族旅行路线。大韩民国安山市作为外国人居住众多的城市，有着以首都圈内最好...  16327   
10  2022-02-10  有谁还没有去过？👀我们的孩子们更喜欢的京畿道利川市必去的旅行路线👨‍👦🌟 雪峰公园代表利川市...  15156   
9   2022-02-15  以家人旅行出名的城市，京畿道杨平郡羊群牧场京畿道杨平郡是包括电影和电视剧中无数登场的著名景点...  15428   
8   2022-02-18  京畿道北部文化与教育中心，介绍一下议政府的代表性旅游景区。议政府是位于京畿道北部，首都的关口...  11812   
7   2022-02-22  世界文化遗产之都,京畿道广州市，自然与历史共存的旅行在京畿道里，广州市是面积比较大的城市。空...   9743   
6   2022-02-24  历史遗迹和现代共存的京畿道光明市，我们家族的旅游路线。京畿道光明市是由于首尔地区变大而产生的...  14879   
5   2022-02-28  京畿道杨州Hot Place，在露营餐厅享受吧！从古至今以美丽风景和新鲜空气而有名的城市，京...   7826   
4   2022-03-02  京畿道东豆川市，在逍遥山相遇的历史中的雪花旅行京畿道东豆川市是 在地缘政治上具有重要意义的地...   8664   
3   2022-03-04  欧洲风格的购物美食街， 始兴市高级折扣店京畿道始兴市有高级折扣店是疲劳的都市人可以得到休息的...   5076   
2   2022-03-08  可以体验游艇和享受大海钓鱼的全谷港玛丽娜京畿道华城市全谷港玛丽娜有着可以享受以辽阔的大海为背...   3912   
1   2022-03-10  走感性时间的旅行地，平泽市风鸟村野游花园花园里吹着微笑风的这个地方是想和珍惜的人一起散步的花...   3058   
0   2022-03-12  华城八景， 济扶落潮和体验神奇海路 济扶岛这是京畿道华城市不可或缺的8个旅游景点之一的岛屿。...   3457   

    좋아요_수  공유수  댓글수   팔로워_수                                                 링크  
25     11    0    4       0  https://weibo.com/6011831795/L8IJk9tQd?from=pa...  
24      3    0    0       0  https://weibo.com/6011831795/L9mdF9t5G?from=pa...  
23      5    0    0       0  https://weibo.com/6011831795/L9wcfEBPu?from=pa...  
22      4    0    1       0  https://weibo.com/6011831795/L9FbKuzBl?from=pa...  
21      5    1    1       0  https://weibo.com/6011831795/LaAyPh9ZV?from=pa...  
20      3    0    0       0  https://weibo.com/6011831795/LaMlxesju?from=pa...  
19      5    1    1       0  https://weibo.com/6011831795/Lbck79XJz?from=pa...  
18      3    1    0       0  https://weibo.com/6011831795/LbCqQmNzF?from=pa...  
17      2    0    0       0  https://weibo.com/6011831795/LcdTRzXmm?from=pa...  
16      3    1    1       0  https://weibo.com/6011831795/Lcyygmjkp?from=pa...  
15      1    0    1       0  https://weibo.com/6011831795/LcBRrAZvy?from=pa...  
14      5    0    1       0  https://weibo.com/6011831795/Ldnc606Rh?from=pa...  
13      3    0    0       0  https://weibo.com/6011831795/LdVR00Ijh?from=pa...  
12      4    0    0       0  https://weibo.com/6011831795/Le88pcEuW?from=pa...  
11      3    0    1       0  https://weibo.com/6011831795/LeurruFMg?from=pa...  
10      4    0    0       0  https://weibo.com/6011831795/LeNtbwHoH?from=pa...  
9       2    1    0       0  https://weibo.com/6011831795/LfuYSiaEG?from=pa...  
8       2    0    1       0  https://weibo.com/6011831795/Lg2AoaSKq?from=pa...  
7       2    0    0       0  https://weibo.com/6011831795/LgF5WsoN4?from=pa...  
6       3    0    0       0  https://weibo.com/6011831795/LgXkk2CxN?from=pa...  
5       1    0    0       0  https://weibo.com/6011831795/LhzKTe0Ht?from=pa...  
4       1    0    0       0  https://weibo.com/6011831795/LhSpqqt1c?from=pa...  
3       2    1    0       0  https://weibo.com/6011831795/LibISiazA?from=pa...  
2       2    0 

In [29]:
# 파일로 출력
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'weibo_' + file_name + '.xlsx'
path = './crawling_data/weibo/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

# 코드 마지막